In [2]:
import requests
import pandas as pd
import os

# ...existing code...
def get_sport_facilities_pandas(api_key):
    """
    Simplify processing using pandas, automatically compatible with different response structures and field names.
    """
    url = "https://api.data.um.edu.mo/service/facilities/sport_complex_facilities/all"
    headers = {
        "Cache-Control": "no-cache",
        "Authorization": api_key,
    }
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()

        # 1) Extract the list of records (compatible with top-level being a dict or list)
        if isinstance(data, list):
            records = data
        elif isinstance(data, dict):
            # Try to find the first list value as the records
            list_candidates = [v for v in data.values() if isinstance(v, list)]
            if not list_candidates:
                raise ValueError(f"API response does not contain a list of records. Available keys: {list(data.keys())}")
            records = list_candidates[0]
        else:
            raise ValueError(f"Unknown return type: {type(data)}")

        # 2) Flatten into a DataFrame
        df = pd.json_normalize(records)

        # 3) Compatible field name mapping
        aliases = {
            "place_code": ["place_code", "placeCode", "place", "placecode"],
            "description": ["description", "desc_eng", "description_en", "desc"],
            "description_chin": ["description_chin", "description_chi", "desc_chi", "description_cn", "description_zh", "desc_zh"],
        }

        # Fill missing columns with NA
        for target, opts in aliases.items():
            src = next((c for c in opts if c in df.columns), None)
            if src is None:
                df[target] = pd.NA
            else:
                if src != target:
                    df[target] = df[src]

        result_df = df[["place_code", "description", "description_chin"]].copy()
        return result_df

    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None

# Usage example
API_KEY = os.getenv('api_key')
facilities_df = get_sport_facilities_pandas(API_KEY)

if facilities_df is not None:
    print(facilities_df)
    # Save to CSV file
    facilities_df.to_csv('../data/sport_facilities.csv', index=False, encoding='utf-8-sig')
    print("\nData has been saved to ../data/sport_facilities.csv")


   place_code                 description description_chin
0        HTT9        Table Tennis Court 9             <NA>
1     HTEN4_6           Tennis Courts 4_6             <NA>
2     HTEN1_3           Tennis Courts 1_3             <NA>
3        HGYM                Fitness Room             <NA>
4        HTTR           Table Tennis Room             <NA>
..        ...                         ...              ...
88      HSPL3        Swimming Pool Lane 3             <NA>
89      HSPL2        Swimming Pool Lane 2             <NA>
90        BA2  Outdoor Basketball Court 2             <NA>
91      HSPL1        Swimming Pool Lane 1             <NA>
92      HSPL0        Swimming Pool Lane 0             <NA>

[93 rows x 3 columns]

Data has been saved to ../data/sport_facilities.csv
